In [1]:
import pyaudio
import os
import struct
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft
import time
from tkinter import TclError
import serial
import mido

# to display in separate Tk window
%matplotlib tk

# constants
CHUNK = 1024*4 # samples per frame
FORMAT = pyaudio.paInt16     # audio format (bytes per sample?)
CHANNELS = 1                 # single channel for microphone
RATE = 44100                 # samples per second

midi_file = r"C:\Users\rajka\Downloads\Queen_-_Bohemian_Rhapsody.mid"
# Set up serial communication with Arduino
arduino = serial.Serial('COM4', 9600)
time.sleep(2)  # Wait for Arduino to initialize

# create matplotlib figure and axes
fig, (ax1, ax2) = plt.subplots(2, figsize=(15, 7))

# pyaudio class instance
p = pyaudio.PyAudio()

# stream object to get data from microphone
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=False,
    output=True,
    frames_per_buffer=CHUNK
)

# variable for plotting
x = np.arange(0, 2 * CHUNK, 2)       # samples (waveform)
xf = np.linspace(0, RATE, CHUNK)     # frequencies (spectrum)

# create a line object with random data
# create a line object with random data
line, = ax1.plot(x, np.random.rand(len(x)), '-', lw=2)

# create semilogx line for spectrum
line_fft, = ax2.semilogx(xf, np.random.rand(len(xf)), '-', lw=2)

# format waveform axes
ax1.set_title('AUDIO WAVEFORM')
ax1.set_xlabel('samples')
ax1.set_ylabel('volume')
ax1.set_ylim(0, 255)
ax1.set_xlim(0, 2 * CHUNK)
plt.setp(ax1, xticks=[0, CHUNK, 2 * CHUNK], yticks=[0, 128, 255])

# format spectrum axes
ax2.set_xlim(20, RATE / 2)
chosen_frequencies = [100, 250, 500, 750, 10000, 2500]
chosen_indices = [np.abs(xf - freq).argmin() for freq in chosen_frequencies]

print('stream started')

# MIDI playback
mid = mido.MidiFile(midi_file)
playback_start_time = time.time()

# MIDI note to frequency conversion function
def note_to_frequency(note):
    return 440.0 * (2.0 ** ((note - 69) / 12.0))

# for measuring frame rate
frame_count = 0
start_time = time.time()

for message in mid.play():
    if message.type == 'note_on':
        # Generate audio data for the note
        frequency = note_to_frequency(message.note)
        duration = message.time
        samples = np.arange(int(RATE * duration))
        wave_data = np.sin(2 * np.pi * frequency * samples / RATE).astype(np.float32)

        # Convert audio data to bytes
        data = (wave_data * 32767).astype(np.int16).tobytes()

        # Write audio data to the stream
        stream.write(data)

        # Convert audio data to integers, make np array, then offset it by 128
        data_int = np.array(struct.unpack(f"{len(data)//2}h", data), dtype=np.int16) + 128


        line.set_ydata(data_int)

        #Compute FFT and update line
        yf = fft(data_int)
        line_fft.set_ydata(np.abs(yf[0:CHUNK]) / (128 * CHUNK))

        # Process chosen frequencies
        for freq in chosen_frequencies:
            index = np.abs(xf - freq).argmin()
            value = np.abs(yf[index]) / (128 * CHUNK)
            print(f"Frequency: {freq} Hz, Value: {value}")
            message = f"{freq}:{value}\n".encode()  # Encode message as bytes
            arduino.write(message)

        # Update figure canvas
        fig.canvas.draw()
        fig.canvas.flush_events()
        frame_count += 1

# Calculate average frame rate and playback time
frame_rate = frame_count / (time.time() - start_time)
playback_time = time.time() - playback_start_time

print('stream stopped')
print(f'average frame rate = {frame_rate:.0f} FPS')
print(f'playback time = {playback_time:.2f} seconds')

# Close the MIDI stream and terminate PyAudio
stream.stop_stream()
stream.close()
p.terminate()

# Show the plot
plt.show()


stream started
Frequency: 100 Hz, Value: 2.243818967716872
Frequency: 250 Hz, Value: 2.092640096594475
Frequency: 500 Hz, Value: 2.190415705324972
Frequency: 750 Hz, Value: 2.3753415161392666
Frequency: 10000 Hz, Value: 3.151626311340267
Frequency: 2500 Hz, Value: 3.0256366320207393


ValueError: invalid number of data points (0) specified